# **507 final project**

First, save the data on google drive and mount to google drive, so we can use the data in colab.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cp -r 'drive/My Drive/507_project/plant2/.' train

In [0]:
from PIL import Image

import random
from matplotlib import pyplot as plt
from os import listdir, path
import os

from torch import nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from glob import glob
import torch
import numpy as np
from torchvision.transforms.transforms import Compose, Resize, ToTensor, CenterCrop
import time
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from timeit import default_timer as timer
import warnings
warnings.filterwarnings('ignore')
import random
import skimage
from skimage import io
from skimage.transform import rotate, AffineTransform, warp
from skimage import util
from skimage.filters import gaussian
from skimage import img_as_ubyte
from torchsummary import summary
from torchvision import datasets, transforms,models
from torch.optim import lr_scheduler
from imblearn.over_sampling import SMOTE

Implement data pre-precessing and split the data into train, test, valid datasets randomly.

In [0]:
def createdir(root_dir, target_dir):
    all_labels = listdir(path.join(root_dir, 'train'))
    all_data = []
    
    # Create a list of tuples out of data samples. Each tuple includes images file name and an int as label code
    for label_code, parent_dir in enumerate(all_labels):
        os.makedirs(path.join(target_dir, parent_dir))


In [0]:
createdir('.','./new/train')
createdir('.','./new/test')
createdir('.','./new/valid')
createdir('.','./new/origin_train')

In [0]:
IMG_SIZE = 256
IMG_SIZE1 = 224

In [0]:
ew_colormin=(25,50,50)
new_colormax=(80,255,255)

def plot_mask(image, colormin, colormax):
        hsv_p1 = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)   #R, G, and B are converted to the floating-point format and scaled to fit the 0 to 1 range
        mask = cv2.inRange(hsv_p1, colormin , colormax)  #?
        result = cv2.bitwise_and(image, image, mask=mask)
        return result


def rotate(image, angle=90, scale=1.0):
    '''
    Rotate the image
    :param image: image to be processed
    :param angle: Rotation angle in degrees. Positive values mean counter-clockwise rotation (the coordinate origin is assumed to be the top-left corner).
    :param scale: Isotropic scale factor.
    '''
    w = image.shape[1]
    h = image.shape[0]
    #rotate matrix
    M = cv2.getRotationMatrix2D((w/2,h/2), angle, scale)
    #rotate
    image = cv2.warpAffine(image,M,(w,h))
    return image


In [0]:
def split_train_test(root_dir, train_dir, val_dir, test_dir, origin_train_dir):
    all_labels = listdir(path.join(root_dir, 'train'))
    all_data = []
    target_size = 1500
    # Create a list of tuples out of data samples. Each tuple includes images file name and an int as label code
    for label_code, parent_dir in enumerate(all_labels):
        name_list = []
        parent_directory = path.join(root_dir, 'train', parent_dir)
        file_list = listdir(parent_directory)
        random.shuffle(file_list)
        for test_sample in file_list[:40]:
            img = cv2.imread(path.join(parent_directory, test_sample))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = plot_mask(img, new_colormin, new_colormax)

            cv2.imwrite(path.join(test_dir, parent_dir, test_sample), img)
        for val_sample in file_list[40:80]:
            img = cv2.imread(path.join(parent_directory, val_sample))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = plot_mask(img, new_colormin, new_colormax)

            cv2.imwrite(path.join(val_dir, parent_dir, val_sample), img)
        origin_size = len(file_list) - 80
        choose = min(int((target_size - origin_size)/3), origin_size)
        for addi in file_list[80:80+choose]:
            angle = np.random.uniform(45,180,1)
            scale = np.random.uniform(0.8,1.2,1)
            img = cv2.imread(path.join(parent_directory, addi))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = plot_mask(img, new_colormin, new_colormax)

            flipped_img = np.fliplr(img)
            flipped_img2 = np.flipud(img)
            rotated_img = rotate(img, angle=angle, scale=scale)
            #noised_img = skimage.util.random_noise(img, mode="poisson")
            cv2.imwrite(path.join(train_dir, parent_dir, addi), img)
            cv2.imwrite(path.join(origin_train_dir, parent_dir, addi), img)

            cv2.imwrite(path.join(train_dir, parent_dir, addi[:len(addi)-4]+"_flip.png"), flipped_img)
            cv2.imwrite(path.join(train_dir, parent_dir, addi[:len(addi)-4]+"_flipud.png"), flipped_img2)
            cv2.imwrite(path.join(train_dir, parent_dir, addi[:len(addi)-4]+"_rotate.png"), rotated_img)
            #io.imsave(path.join(train_dir, parent_dir, addi[:len(addi)-4]+"_noise.png"), img_as_ubyte(noised_img))
        for rest in file_list[80+choose:]:
            img = cv2.imread(path.join(parent_directory, rest))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = plot_mask(img, new_colormin, new_colormax)

            cv2.imwrite(path.join(origin_train_dir, parent_dir, rest), img)
            cv2.imwrite(path.join(train_dir, parent_dir, rest), img)

        if 4*origin_size < target_size:
            choose2 = min(int(target_size-4*origin_size), origin_size)
            for add2 in file_list[80:80+choose2]:
                angle = np.random.uniform(45,180,1)
                scale = np.random.uniform(0.8,1.2,1)
                img = cv2.imread(path.join(parent_directory, add2))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = plot_mask(img, new_colormin, new_colormax)
                rotated_img2 = rotate(img, angle=angle, scale=scale)
                cv2.imwrite(path.join(train_dir, parent_dir, add2[:len(add2)-4]+"_rotate2.png"), rotated_img2)

        if 5*origin_size < target_size:
            choose3 = min(int(target_size-5*origin_size), origin_size)
            for add3 in file_list[80:80+choose3]:
                angle = np.random.uniform(45,180,1)
                scale = np.random.uniform(0.8,1.2,1)
                img = cv2.imread(path.join(parent_directory, add3))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = plot_mask(img, new_colormin, new_colormax)
                rotated_img3 = rotate(img, angle=angle, scale=scale)
                cv2.imwrite(path.join(train_dir, parent_dir, add3[:len(add3)-4]+"_rotate3.png"), rotated_img3)

        if 6*origin_size < target_size:
            choose4 = min(int(target_size-6*origin_size), origin_size)
            for add4 in file_list[80:80+choose4]:
                angle = np.random.uniform(45,180,1)
                scale = np.random.uniform(0.8,1.2,1)
                img = cv2.imread(path.join(parent_directory, add4))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = plot_mask(img, new_colormin, new_colormax)
                rotated_img4 = rotate(img, angle=angle, scale=scale)
                cv2.imwrite(path.join(train_dir, parent_dir, add4[:len(add4)-4]+"_rotate4.png"), rotated_img4)

In [0]:
split_train_test('.', './new/train','./new/valid', './new/test', './new/origin_train')

In [0]:
def getDataLoaders_split(train_dir, train_transforms, batch_size):
    all_labels = listdir(train_dir)
    all_data = []
    for label_code, parent_dir in enumerate(all_labels):
        this_dir_images = glob(path.join(train_dir, parent_dir, '*.png'))
        all_data += zip(this_dir_images, [label_code]*len(this_dir_images))
    random.shuffle(all_data)
    train_data = SeedlingDataset(all_data, len(all_labels), transform=train_transforms)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=0)
    return train_loader

The first model we use is a 4-layer CNN model. We bulid the model and train it.

In [0]:
class TinyModel(nn.Module):
    def __init__(self):
        super(TinyModel, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(3, 12, kernel_size = 5, stride = 1,padding = 2, bias = True), nn.BatchNorm2d(12), nn.ReLU())
        self.conv2 = nn.Sequential(nn.Conv2d(12, 24, kernel_size = 5, stride = 1, bias = True), nn.BatchNorm2d(24), nn.ReLU())
        self.conv3 = nn.Sequential(nn.Conv2d(24, 48, kernel_size = 5, stride = 1, bias = True), nn.BatchNorm2d(48), nn.ReLU())
        self.conv4 = nn.Sequential(nn.Conv2d(48, 96, kernel_size = 5, stride = 1, bias = True), nn.BatchNorm2d(96), nn.ReLU())

        self.fc1 = nn.Linear(in_features=9600, out_features=1200)
        self.fc2 = nn.Linear(in_features=1200, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=12)
        self.pool=nn.MaxPool2d(2,stride=2)

    def forward(self, x):
        #print(x.shape) 
        x = self.pool(self.conv1(x))
        x = self.pool(self.conv2(x))
        x = self.pool(self.conv3(x))
        x = self.pool(self.conv4(x))
        # print(x.shape)
        x = x.view(x.size(0), -1)
        #print(x.shape)
        x = F.relu(self.fc1(x))
        # print(x.shape)
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return F.log_softmax(x, dim=1)

    def loss(self, prediction, true_values):

        return F.nll_loss(prediction, true_values)

In [0]:
# define a class for the dataset
class SeedlingDataset(Dataset):
    
    def __init__(self, data, n_labels, transform=None):
        
        self.data = data
        self.transform = transform
        self.n_labels = n_labels
        self.transform = transform
        
    def __len__(self):
        
        return len(self.data)
    
    def __getitem__(self, idx):
        
        data_file, image_code = self.data[idx]
        img = Image.open(data_file)
        img = img.convert('RGB')
        if self.transform:
            img = self.transform(img)
        img = torch.from_numpy(np.array(img))
        label = torch.tensor(image_code)
        
        return img, label
    

In [0]:
def train(model, device, train_dataloader, optimizer, epoch, verbose=False):
    
    model.train()
    total_loss = 0
    for batch_idx, (image, label) in enumerate(train_dataloader):
        
        input_var = image.to(device)
        target_var = label.to(device)
        optimizer.zero_grad()
        output = model(input_var)
        loss = model.loss(output, target_var)
        loss.backward()
        optimizer.step()
        total_loss += loss
        if batch_idx % 10 == 0 and verbose:
            print('Train Epoch: {0}, Train batch: {1}, Batch loss: {2}'
                  .format(epoch, batch_idx, loss))
    
    epoch_loss = total_loss/(batch_idx + 1)
    
    return epoch_loss
    
def valid(model, device, dataloader):
    
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch_idx, (image, label) in enumerate(dataloader):
          input_var = image.to(device)
          target_var = label.to(device)
          output = model(input_var)
          loss = model.loss(output, target_var)
          total_loss += loss
      
        epoch_loss = total_loss/(batch_idx + 1)
        # print('validation loss: ',epoch_loss)
    return epoch_loss
    
def test(model, device, test_dataloader):
    
    model.eval()
    correct = 0
    with torch.no_grad():
        for image, label in test_dataloader:
            input_var = image.to(device)
            target_var = label.to(device)
            output = model(input_var)
            prediction = output.argmax(dim=1, keepdim=True)
            correct += prediction.eq(target_var.view_as(prediction)).sum().item()
        accuracy = correct / len(test_dataloader.dataset)
    
    return accuracy

In [0]:
def AdjustLR(optim, lr):
    '''
    This is a simple hand-made function to decrease 
    optimizer's learning rate by a factor of 10 at a specific time
    '''
    for param_group in optim.param_groups:
            param_group['lr'] /= 10

In [0]:
losses = []
valid_losses = []
accuracies = []
test_accuracies = []
n_epochs = 20
learning_rate = 0.05
lr_decay = 5 # every 10 epochs, the learning rate is divided by 10
batch_size = 64
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

use_cuda = True # use True to switch to GPU

device = torch.device("cuda" if use_cuda else "cpu")
model = TinyModel().to(device)
print('The model has {0} parameters'.format(sum([len(i.reshape(-1)) for i in model.parameters()]) ))

In [0]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

mean1 = [0.5, 0.5, 0.5]
std1 = [0.5, 0.5, 0.5]

test_transform = Compose([Resize(IMG_SIZE), CenterCrop(IMG_SIZE1), ToTensor(), transforms.Normalize(mean, std)])
train_transform = Compose([
    transforms.RandomResizedCrop(size=IMG_SIZE, scale=(0.8, 1.0)),
    transforms.RandomRotation(degrees=15),
    transforms.CenterCrop(size=IMG_SIZE1),  # Image net standards
    transforms.ToTensor(),
    transforms.Normalize(mean, std)  # Imagenet standards
])

# valid_data = SeedlingDataset(valid_data_filename, len(all_labels), transform=test_transform)
valid_loader = getDataLoaders_split('./new/valid', test_transform, batch_size = batch_size)

# train_data = SeedlingDataset(train_data_filename, len(all_labels), transform=train_transform)
train_loader = getDataLoaders_split('./new/train', train_transform, batch_size = batch_size)

# test_data = SeedlingDataset(test_data_filename, len(all_labels), transform=test_transform)
test_loader = getDataLoaders_split('./new/test', test_transform, batch_size = batch_size)

print('number of train examples: {0}, number of validation examples: {1}, number of test examples: {2}'
      .format(len(train_loader.dataset), len(test_loader.dataset), len(test_loader.dataset)))

In [0]:
for epoch in range(1, n_epochs + 1):
    tic = time.time()
    epoch_loss = train(model, device, train_loader, optimizer, epoch)
    print('Training loss for epoch {0} is {1:.5f}'.format(epoch, epoch_loss))
    losses.append(epoch_loss)
    valid_loss = valid(model, device, valid_loader)
    valid_losses.append(valid_loss)
    print('Val loss: {0:.3f}'.format(valid_loss))
    test_accuracy = test(model, device, test_loader)
    print('Test accuracy: {0:.3f}'.format(test_accuracy))
    test_accuracies.append(test_accuracy)
    tac = time.time()
    print('Epoch time: {0:0.1f} seconds'.format(tac - tic))
    if epoch % lr_decay == 0:
        AdjustLR(optimizer, learning_rate)


Evaluate the 4-layer CNN model based on loss and accuracy plot; confusion matrix and f1 score.

In [0]:
import matplotlib.pyplot as plt
fig, ax1 = plt.subplots()

step = np.arange(1,21, dtype = int)
color = 'tab:red'
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Average Cross Entropy', color=color)
ax1.plot(step, losses, label="Train loss")
ax1.plot(step,valid_losses, label="Valid loss")
ax1.tick_params(axis='y', labelcolor=color)

# ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

# color = 'tab:blue'
# ax2.set_ylabel('Accuracy', color=color)
# # ax2.set_ylim(0,1)  # we already handled the x-label with ax1
# ax2.plot(step, test_accuracies, color="g", label = "Test Accuracy")
# ax2.tick_params(axis='y', labelcolor=color)


ax1.legend(loc=10)
# ax2.legend(loc=7)
plt.title('Validation& Training losses')
# fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [0]:
from sklearn.metrics import f1_score, precision_score, recall_score

# model.eval()
# nb_classes = 12
# confusion_matrix = torch.zeros(nb_classes, nb_classes)
# with torch.no_grad():
#     for i, (inputs, classes) in enumerate(test_loader):
#         inputs = inputs.to(device)
#         classes = classes.to(device)
#         outputs = model(inputs)
#         _, preds = torch.max(outputs, 1)
#         for t, p in zip(classes.view(-1), preds.view(-1)):
#             confusion_matrix[t.long(), p.long()] += 1
# print(confusion_matrix)
recall = confusion_matrix2.diag()/confusion_matrix2.sum(1) #recall
precision = confusion_matrix2.diag()/confusion_matrix2.sum(0) #precision
f1 = f1_score(classes.cpu().numpy(), preds.cpu().numpy(), average='macro')
p = precision_score(classes.cpu().numpy(), preds.cpu().numpy(), average="macro")
r = recall_score(classes.cpu().numpy(), preds.cpu().numpy(), average="macro")
print(f1, p, r)
confusion_matrix1 = torch.zeros(nb_classes, nb_classes)


In [0]:
import pandas as pd
import seaborn as sns

# CATEGORIES = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent',
#               'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']

# abbreviation = ['BG', 'Ch', 'Cl', 'CC', 'CW', 'FH', 'LSB', 'M', 'SM', 'SP', 'SFC', 'SB']
CATEGORIES = ['Common Chickweed',
 'Small-flowered Cranesbill',
 'Maize',
 'Loose Silky-bent',
 'Black-grass',
 'Common wheat',
 'Scentless Mayweed',
 'Cleavers',
 'Charlock',
 'ShepherdGÇÖs Purse',
 'Fat Hen',
 'Sugar beet']
abbreviation = ['CC', 'SFC', 'M', 'LSB', 'BG', 'CW', 'SM', 'Cl', 'Ch', 'SP', 'FH', 'SB']
pd.DataFrame({'class': CATEGORIES, 'abbreviation': abbreviation})
fig, ax = plt.subplots(1)
ax = sns.heatmap(confusion_matrix2, ax=ax, cmap=plt.cm.Blues, annot=True)
ax.set_xticklabels(abbreviation)
ax.set_yticklabels(abbreviation)
plt.title('Confusion Matrix')
plt.ylabel('True class')
plt.xlabel('Predicted class')
#fig.savefig('Confusion matrix.png', dpi=300)
plt.show()

Fit the pre-trained model

In [0]:
drive.mount('/content/drive')
path = "/content/drive/My Drive/plant_seedlings_classification"

os.chdir(path)
os.listdir(path)

In [0]:
image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224), 
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    # Validation 
    'val':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    # Test
    'test':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [0]:

batch_size = 64
data = {
    'train':
    datasets.ImageFolder(root='datasplitted1/train', transform=image_transforms['train']),
    'val':
    datasets.ImageFolder(root='datasplitted1/val', transform=image_transforms['val']),
    'test':
    datasets.ImageFolder(root='datasplitted1/test', transform=image_transforms['test'])
}

# Dataloader iterators
dataloaders = {
    'train': DataLoader(data['train'], batch_size=batch_size, shuffle=True),
    'val': DataLoader(data['val'], batch_size=batch_size, shuffle=True),
    'test': DataLoader(data['test'], batch_size=batch_size, shuffle=True)
}


In [0]:
# model = models.vgg16(pretrained=True)
model = models.resnet152(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

In [0]:
n_class = len(data['train'].classes)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
            nn.Linear(num_ftrs, 256), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(256, n_class), nn.LogSoftmax(dim=1))

In [0]:
model = model.to('cuda')
summary(model, input_size=(3, 224, 224), batch_size=batch_size, device='cuda')

In [0]:
model.class_to_idx = data['train'].class_to_idx
model.idx_to_class = {
    idx: class_
    for class_, idx in model.class_to_idx.items()
}

list(model.idx_to_class.items())

In [0]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters())

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [0]:
save_file_name = 'resnet152-transfer-3.pt'
checkpoint_path = 'resnet152-transfer-3.pth'
def train(model,
          criterion,
          optimizer,
          train_loader,
          valid_loader,
          save_file_name,
          max_epochs_stop=3,
          n_epochs=20,
          print_every=2):
    """Train a PyTorch Model

    Params
    --------
        model (PyTorch model): cnn to train
        criterion (PyTorch loss): objective to minimize
        optimizer (PyTorch optimizier): optimizer to compute gradients of model parameters
        train_loader (PyTorch dataloader): training dataloader to iterate through
        valid_loader (PyTorch dataloader): validation dataloader used for early stopping
        save_file_name (str ending in '.pt'): file path to save the model state dict
        max_epochs_stop (int): maximum number of epochs with no improvement in validation loss for early stopping
        n_epochs (int): maximum number of training epochs
        print_every (int): frequency of epochs to print training stats

    Returns
    --------
        model (PyTorch model): trained cnn with best weights
        history (DataFrame): history of train and validation loss and accuracy
    """

    # Early stopping intialization
    epochs_no_improve = 0
    valid_loss_min = np.Inf

    valid_max_acc = 0
    history = []

    # Number of epochs already trained (if using loaded in model weights)
    try:
        print(f'Model has been trained for: {model.epochs} epochs.\n')
    except:
        model.epochs = 0
        print(f'Starting Training from Scratch.\n')

    overall_start = timer()

    # Main loop
    for epoch in range(n_epochs):

        # keep track of training and validation loss each epoch
        train_loss = 0.0
        valid_loss = 0.0

        train_acc = 0
        valid_acc = 0

        # Set to training
        model.train()
        start = timer()

        # Training loop
        for ii, (data, target) in enumerate(train_loader):
            # Tensors to gpu
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()

            # Clear gradients
            optimizer.zero_grad()
            # Predicted outputs are log probabilities
            output = model(data)

            # Loss and backpropagation of gradients
            loss = criterion(output, target)
            loss.backward()

            # Update the parameters
            optimizer.step()

            # Track train loss by multiplying average loss by number of examples in batch
            train_loss += loss.item() * data.size(0)

            # Calculate accuracy by finding max log probability
            _, pred = torch.max(output, dim=1)
            correct_tensor = pred.eq(target.data.view_as(pred))
            # Need to convert correct tensor from int to float to average
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
            # Multiply average accuracy times the number of examples in batch
            train_acc += accuracy.item() * data.size(0)

            # Track training progress
            print(
                f'Epoch: {epoch}\t{100 * (ii + 1) / len(train_loader):.2f}% complete. {timer() - start:.2f} seconds elapsed in epoch.',
                end='\r')

        # After training loops ends, start validation
        else:
            model.epochs += 1

            # Don't need to keep track of gradients
            with torch.no_grad():
                # Set to evaluation mode
                model.eval()

                # Validation loop
                for data, target in valid_loader:
                    # Tensors to gpu
                    if train_on_gpu:
                        data, target = data.cuda(), target.cuda()

                    # Forward pass
                    output = model(data)

                    # Validation loss
                    loss = criterion(output, target)
                    # Multiply average loss times the number of examples in batch
                    valid_loss += loss.item() * data.size(0)

                    # Calculate validation accuracy
                    _, pred = torch.max(output, dim=1)
                    correct_tensor = pred.eq(target.data.view_as(pred))
                    accuracy = torch.mean(
                        correct_tensor.type(torch.FloatTensor))
                    # Multiply average accuracy times the number of examples
                    valid_acc += accuracy.item() * data.size(0)

                # Calculate average losses
                train_loss = train_loss / len(train_loader.dataset)
                valid_loss = valid_loss / len(valid_loader.dataset)

                # Calculate average accuracy
                train_acc = train_acc / len(train_loader.dataset)
                valid_acc = valid_acc / len(valid_loader.dataset)

                history.append([train_loss, valid_loss, train_acc, valid_acc])

                # Print training and validation results
                if (epoch + 1) % print_every == 0:
                    print(
                        f'\nEpoch: {epoch} \tTraining Loss: {train_loss:.4f} \tValidation Loss: {valid_loss:.4f}'
                    )
                    print(
                        f'\t\tTraining Accuracy: {100 * train_acc:.2f}%\t Validation Accuracy: {100 * valid_acc:.2f}%'
                    )

                # Save the model if validation loss decreases
                if valid_loss < valid_loss_min:
                    # Save model
                    torch.save(model.state_dict(), save_file_name)
                    # Track improvement
                    epochs_no_improve = 0
                    valid_loss_min = valid_loss
                    valid_best_acc = valid_acc
                    best_epoch = epoch

                # Otherwise increment count of epochs with no improvement
                else:
                    epochs_no_improve += 1
                    # Trigger early stopping
                    if epochs_no_improve >= max_epochs_stop:
                        print(
                            f'\nEarly Stopping! Total epochs: {epoch}. Best epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%'
                        )
                        total_time = timer() - overall_start
                        print(
                            f'{total_time:.2f} total seconds elapsed. {total_time / (epoch+1):.2f} seconds per epoch.'
                        )

                        # Load the best state dict
                        model.load_state_dict(torch.load(save_file_name))
                        # Attach the optimizer
                        model.optimizer = optimizer

                        # Format history
                        history = pd.DataFrame(
                            history,
                            columns=[
                                'train_loss', 'valid_loss', 'train_acc',
                                'valid_acc'
                            ])
                        return model, history

    # Attach the optimizer
    model.optimizer = optimizer
    # Record overall time and print out stats
    total_time = timer() - overall_start
    print(
        f'\nBest epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%'
    )
    print(
        f'{total_time:.2f} total seconds elapsed. {total_time / (epoch):.2f} seconds per epoch.'
    )
    # Format history
    history = pd.DataFrame(
        history,
        columns=['train_loss', 'valid_loss', 'train_acc', 'valid_acc'])
    return model, history

In [0]:
train_on_gpu = True
model, history = train(
    model,
    criterion,
    optimizer,
    dataloaders['train'],
    dataloaders['val'],
    save_file_name=save_file_name,
    max_epochs_stop=5,
    n_epochs=40,
    print_every=2)

Evaluate the pretrained model

In [0]:
# plt.figure(figsize=(16, 12))
fig, ax1 = plt.subplots()
color = 'tab:red'
for c in ['train_loss', 'valid_loss']:
    ax1.plot(
        history[c], label=c)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Average Cross Entropy')
ax1.tick_params(axis='y', labelcolor=color)


ax2 = ax1.twinx()
color = 'tab:blue'
# for c in ['train_acc', 'valid_acc']:
ax2.plot(history['train_acc'], label='train_acc', color='r')
ax2.plot(history['valid_acc'], label='valid_acc', color='g')
ax2.set_ylabel('Average Accuracy')
ax2.tick_params(axis='y', labelcolor=color)

ax1.legend(loc = 10)
ax2.legend(loc = 7)
# plt.xlabel('Epoch')
# plt.ylabel('Average Cross Entropy')
plt.title('Training and Validation Losses & Accuracy')

In [0]:
plt.figure(figsize=(8, 6))
for c in ['train_acc', 'valid_acc']:
    plt.plot(
        100 * history[c], label=c)
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Average Accuracy')
plt.title('Training and Validation Accuracy')

In [0]:
from sklearn.metrics import confusion_matrix
import itertools
def cm_acc_ontest(model, test_data, device='cuda'):
    # Initialize the prediction and label lists(tensors)
    predlist=torch.zeros(0,dtype=torch.long)
    lbllist=torch.zeros(0,dtype=torch.long)

    with torch.no_grad():
        model.eval()
        for i, (data, target) in enumerate(test_data):
            data = data.to(device)
            target = target.to(device)
            outputs = model(data)
            _, preds = torch.max(outputs, 1)

            # Append batch prediction results
            predlist=torch.cat([predlist,preds.view(-1).cpu()])
            lbllist=torch.cat([lbllist,target.view(-1).cpu()])

    # Confusion matrix
    conf_mat=confusion_matrix(lbllist.numpy(), predlist.numpy())

    # Per-class accuracy
    class_accuracy=100*conf_mat.diagonal()/conf_mat.sum(1)
    return conf_mat, class_accuracy

# https://deeplizard.com/learn/video/0LhiS6yu2qQ
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Greens):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    plt.figure(figsize=(10,10))
    # print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
classes = [
    class_
    for class_, idx in model.class_to_idx.items()
]
cm, acc = cm_acc_ontest(model, dataloaders['test'])
plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues)


Additional Steps. Use both model A and model B.

In [ ]:
def getDataLoaders_split2(train_dir, train_transforms, batch_size, randomsh = True):
    all_labels = listdir(train_dir)
    all_data = []
    for label_code, parent_dir in enumerate(all_labels):
        this_dir_images = glob(path.join(train_dir, parent_dir, '*.png'))
        all_data += zip(this_dir_images, [label_code]*len(this_dir_images))
    if randomsh:
        random.shuffle(all_data)
    train_data = SeedlingDataset(all_data, len(all_labels), transform=train_transforms)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=randomsh, num_workers=0)
    return train_loader
# main
from torchvision import datasets, transforms,models


mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

mean1 = [0.5, 0.5, 0.5]
std1 = [0.5, 0.5, 0.5]

test_transform = Compose([Resize(IMG_SIZE), CenterCrop(IMG_SIZE1), ToTensor(), transforms.Normalize(mean, std)])


IMG_SIZE = 256
IMG_SIZE1 = 224

batch_size = 64
use_cuda = True # use True to switch to GPU
device = torch.device("cuda" if use_cuda else "cpu")
model = TinyModel().to(device)

model.load_state_dict(torch.load('D:/Yueyang/model_save/4_layer_model.pt'))
model.eval() # model A

nb_classes=12
test_loader2 = getDataLoaders_split2('D:/yueyang/test', test_transform, batch_size = 480, randomsh = False)
confusion_matrix_A = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(test_loader2):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        # Team member has different order, so need to resort.
        preds[preds == 4] = 100+0
        preds[preds == 8] = 100+1
        preds[preds == 7] = 100+2
        preds[preds == 0] = 100+3
        preds[preds == 5] = 100+4
        preds[preds == 10] = 100+5
        preds[preds == 3] = 100+6
        preds[preds == 2] = 100+7
        preds[preds == 6] = 100+8
        preds[preds == 9] = 100+9
        preds[preds == 1] = 100+10
        preds[preds == 11] = 100+11

        preds = preds-100
     
        for t, p in zip(classes.view(-1), preds.view(-1)):
            confusion_matrix_A[t.long(), p.long()] += 1
print(confusion_matrix_A)

f1 = f1_score(classes.cpu().numpy(), preds.cpu().numpy(), average='macro')
p = precision_score(classes.cpu().numpy(), preds.cpu().numpy(), average="macro")
r = recall_score(classes.cpu().numpy(), preds.cpu().numpy(), average="macro")
print(f1, p, r)

print(f1_score(classes.cpu(), preds.cpu(), average=None))
print(precision_score(classes.cpu(), preds.cpu(), average=None))
print(recall_score(classes.cpu(), preds.cpu(), average=None))

#4,8,7,0,5,10,3,2,6,9,1,11
#['Common Chickweed'0, 'Small-flowered Cranesbill'1, 'Maize'2, 'Loose Silky-bent'3, 'Black-grass'4, 'Common wheat'5, 
# 'Scentless Mayweed'6, 'Cleavers'7, 'Charlock'8, 'ShepherdGÇÖs Purse'9, 'Fat Hen'10, 'Sugar beet'11]

In [ ]:
pp = np.array(preds.cpu()).tolist()
index_both = [index for index in range(len(pp)) if pp[index] == 0 or pp[index] == 6]

model.load_state_dict(torch.load('D:/Yueyang/model_save/2_modle_10.pth'))
model.eval()# model B

correct = 0
with torch.no_grad():
    input_var = inputs[index_both]
    target_var = classes[index_both]
    output = model(input_var)
    _, prediction = torch.max(output, 1)
    correct = (target_var == prediction).sum().item()
    accuracy = correct / input_var.shape[0]
accuracy



prediction[prediction == 1] = 6
prediction[prediction == 0] = 0

preds[index_both] = prediction

In [ ]:
confusion_matrix_B = torch.zeros(nb_classes, nb_classes)

for t, p in zip(classes.view(-1), preds.view(-1)):
    confusion_matrix_B[t.long(), p.long()] += 1
print(confusion_matrix_B)
print(confusion_matrix_B.diag()/confusion_matrix_B.sum(1))


from sklearn.metrics import f1_score, precision_score, recall_score
f1 = f1_score(classes.cpu(), preds.cpu(), average='macro')
p = precision_score(classes.cpu(), preds.cpu(), average='macro')
r = recall_score(classes.cpu(), preds.cpu(), average='macro')
print(f1) # another measure.
print(p) # another measure.
print(r) # another measure.

print(f1_score(classes.cpu(), preds.cpu(), average=None))
print(precision_score(classes.cpu(), preds.cpu(), average=None))
print(recall_score(classes.cpu(), preds.cpu(), average=None))





f1 = f1_score(classes.cpu().numpy(), preds.cpu().numpy(), average='macro')
p = precision_score(classes.cpu().numpy(), preds.cpu().numpy(), average="macro")
r = recall_score(classes.cpu().numpy(), preds.cpu().numpy(), average="macro")
print(f1, p, r)

In [ ]:
import pandas as pd
import seaborn as sns

CATEGORIES = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent',
              'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']

abbreviation = ['BG', 'Ch', 'Cl', 'CC', 'CW', 'FH', 'LSB', 'M', 'SM', 'SP', 'SFC', 'SB']
pd.DataFrame({'class': CATEGORIES, 'abbreviation': abbreviation})
fig, ax = plt.subplots(1)
ax = sns.heatmap(confusion_matrix_B, ax=ax, cmap=plt.cm.Blues, annot=True)
ax.set_xticklabels(abbreviation)
ax.set_yticklabels(abbreviation)
plt.title('Confusion Matrix')
plt.ylabel('True class')
plt.xlabel('Predicted class')
fig.savefig('D:/507_plant_classification/cm_modelB.png', dpi=300)
plt.show()


The following is to get model B 

In [ ]:
def image_augmentation(root_dir):
    all_labels = listdir(path.join(root_dir))
    all_data = []
    
    # Create a list of tuples out of data samples. Each tuple includes images file name and an int as label code
    for label_code, parent_dir in enumerate(all_labels):
        parent_directory = path.join(root_dir, parent_dir)
        file_list = listdir(parent_directory)
        for file_name in file_list:
            img = cv2.imread(path.join(parent_directory, file_name))
            for i in range(36):
                angle = 180/36*i + 16
                scale = np.random.uniform(0.8,1.2,1)
                rotated_img = rotate(img, angle=angle, scale=scale)
                cv2.imwrite(path.join(parent_directory, file_name[:len(file_name)-4]+"_rotate"+str(i)+".png"), rotated_img)
                
#image_augmentation('D:/Yueyang/new_train')

In [ ]:
from torchvision import datasets, transforms,models

IMG_SIZE = 256
IMG_SIZE1 = 224


losses = []
val_accuracies = []
test_accuracies = []

n_epochs = 20
learning_rate = 0.05
lr_decay = 5 # every 10 epochs, the learning rate is divided by 10
batch_size = 64
use_cuda = True # use True to switch to GPU

device = torch.device("cuda" if use_cuda else "cpu")
model = TinyModel().to(device)
print('The model has {0} parameters'.format(sum([len(i.reshape(-1)) for i in model.parameters()]) ))



mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

mean1 = [0.5, 0.5, 0.5]
std1 = [0.5, 0.5, 0.5]

test_transform = Compose([Resize(IMG_SIZE), CenterCrop(IMG_SIZE1), ToTensor(), transforms.Normalize(mean, std)])

train_loader = getDataLoaders_split('D:/yueyang/new_train',test_transform, batch_size = batch_size)
test_loader = getDataLoaders_split('D:/yueyang/2_test', test_transform, batch_size = batch_size)
val_loader = getDataLoaders_split('D:/yueyang/new_val', test_transform, batch_size = batch_size)


print('number of train examples: {0}, number of test examles: {1}'
      .format(len(train_loader.dataset), len(test_loader.dataset)))

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(1, n_epochs + 1):
    tic = time.time()
    epoch_loss = train(model, device, train_loader, optimizer, epoch)
    print('Training loss for epoch {0} is {1:.5f}'.format(epoch, epoch_loss))
    losses.append(epoch_loss)
    val_accuracy = test(model, device, val_loader)
    print('Val accuracy: {0:.3f}'.format(val_accuracy))
    
    val_accuracies.append(val_accuracy)
    tac = time.time()
    print('Epoch time: {0:0.1f} seconds'.format(tac - tic))
    if val_accuracy>0.7:
        torch.save(model.state_dict(), "D:/0Yueyang/model_save/2_modle_"+str(epoch)+".pth")

    if epoch % lr_decay == 0:
        AdjustLR(optimizer, learning_rate)

        
# use the complicated model, bs = 64, origin data, 82.5%
# use the same model, bs = 32, aug data, 86.5%


updated pretrained model part codes：

In [ ]:
from torchvision import transforms
import pandas as pd
import matplotlib.pyplot as plt
from os import listdir, path
import cv2
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import models
import torch.nn as nn
from torch import optim, cuda
from torchsummary import summary
import numpy as np
from timeit import default_timer as timer
import torch
from torch.optim import lr_scheduler
from PIL import Image

import split_folders
split_folders.fixed('train', output="datasplitted1", seed=1337, fixed=(40, 40), oversample=False)

new_colormin=(25,50,50)
new_colormax=(80,255,255)

def plot_mask(image, colormin, colormax):
        hsv_p1 = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)   
        mask = cv2.inRange(hsv_p1, colormin , colormax)
        result = cv2.bitwise_and(image, image, mask=mask)
        #plt.figure(figsize=(15,10))
        #plt.subplot(1, 3, 1)
        #plt.imshow(image)
        #plt.grid(False)
        #plt.subplot(1, 3, 2)
        #plt.imshow(mask, cmap="gray")
        #plt.grid(False)
        #plt.subplot(1, 3, 3)
        #plt.imshow(result)
        #plt.grid(False)
        return result

def color_seg_rewrite(root_dir):
    all_labels = listdir(path.join(root_dir))
    all_data = []
    
    # Create a list of tuples out of data samples. Each tuple includes images file name and an int as label code

    for label_code, parent_dir in enumerate(all_labels):
        print(parent_dir)
        parent_directory = path.join(root_dir, parent_dir)
        file_list = listdir(parent_directory)
        for file_name in file_list:
            print(file_name)
            img = cv2.imread(path.join(parent_directory, file_name))
            
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = plot_mask(img, new_colormin, new_colormax)
            
            cv2.imwrite(path.join(parent_directory, file_name), img)
color_seg_rewrite('datasplitted1/train')
color_seg_rewrite('datasplitted1/test')
color_seg_rewrite('datasplitted1/val')

image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=30),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224), 
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    # Validation 
    'val':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    # Test
    'test':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

batch_size = 16
data = {
    'train':
    datasets.ImageFolder(root='datasplitted1/train', transform=image_transforms['train']),
    'val':
    datasets.ImageFolder(root='datasplitted1/val', transform=image_transforms['val']),
    'test':
    datasets.ImageFolder(root='datasplitted1/test', transform=image_transforms['test'])
}

# Dataloader iterators
dataloaders = {
    'train': DataLoader(data['train'], batch_size=batch_size, shuffle=True),
    'val': DataLoader(data['val'], batch_size=batch_size, shuffle=True),
    'test': DataLoader(data['test'], batch_size=batch_size, shuffle=True)
}

model = models.resnet152(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

n_class = len(data['train'].classes)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
            nn.Linear(num_ftrs, 256), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(256, n_class), nn.LogSoftmax(dim=1))
model = model.to('cuda')
model.class_to_idx = data['train'].class_to_idx
model.idx_to_class = {
    idx: class_
    for class_, idx in model.class_to_idx.items()
}

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters())

save_file_name = 'resnet152-transfer-1.pt'
checkpoint_path = 'resnet152-transfer-1.pth'
def train(model,
          criterion,
          optimizer,
          scheduler,
          train_loader,
          valid_loader,
          save_file_name,
          max_epochs_stop=3,
          n_epochs=30,
          device='cuda'):
   

    valid_loss_min = np.Inf

    valid_max_acc = 0
    history = []

    # Number of epochs already trained (if using loaded in model weights)
    try:
        print(f'Model has been trained for: {model.epochs} epochs.\n')
    except:
        model.epochs = 0
        print(f'Starting Training from Scratch.\n')

    overall_start = timer()

    # Main loop
    for epoch in range(n_epochs):

        # keep track of training and validation loss each epoch
        train_loss = 0.0
        valid_loss = 0.0

        train_acc = 0
        valid_acc = 0

        # Set to training
        model.train()
        start = timer()

        # Iterate over data.
        for ii, (data, target) in enumerate(train_loader):
            model.train()
            data, target = data.to(device), target.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()
            # forward
            output = model(data)
            _, pred = torch.max(output, dim=1)
            loss = criterion(output, target)
            
            # backward
            loss.backward()
            optimizer.step()

            # stats
            train_loss += loss.item() * data.size(0)
            correct_tensor = pred.eq(target.data.view_as(pred))
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
            train_acc += accuracy.item() * data.size(0)

            # Track training progress
            print(
                f'Epoch: {epoch}\t{100 * (ii + 1) / len(train_loader):.2f}% complete. {timer() - start:.2f} seconds elapsed in epoch.',
                end='\r')

        # After training loops ends, start validation
        else:
            model.epochs += 1
            # Don't need to keep track of gradients
            with torch.no_grad():
                # Set to evaluation mode
                model.eval()

                # Validation loop
                for data, target in valid_loader:
                    # Tensors to gpu
                    data, target = data.to(device), target.to(device)

                    # Forward pass
                    output = model(data)
                    _, pred = torch.max(output, dim=1)
                    loss = criterion(output, target)

                    # stats
                    valid_loss += loss.item() * data.size(0)
                    correct_tensor = pred.eq(target.data.view_as(pred))
                    accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
                    valid_acc += accuracy.item() * data.size(0)

                # Calculate average losses
                train_loss = train_loss / len(train_loader.dataset)
                valid_loss = valid_loss / len(valid_loader.dataset)

                # Calculate average accuracy
                train_acc = train_acc / len(train_loader.dataset)
                valid_acc = valid_acc / len(valid_loader.dataset)

                history.append([train_loss, valid_loss, train_acc, valid_acc])
                scheduler.step()

                
                print(
                    f'\nEpoch: {epoch} \tTraining Loss: {train_loss:.4f} \tValidation Loss: {valid_loss:.4f}'
                )
                print(
                    f'\t\tTraining Accuracy: {100 * train_acc:.2f}%\t Validation Accuracy: {100 * valid_acc:.2f}%'
                )

                # Save the model if validation loss decreases
                if valid_loss < valid_loss_min:
                    # Save model
                    torch.save(model.state_dict(), save_file_name)
                    # Track improvement
                    epochs_no_improve = 0
                    valid_loss_min = valid_loss
                    valid_best_acc = valid_acc
                    best_epoch = epoch

    # Attach the optimizer
    model.optimizer = optimizer
    # Record overall time and print out stats
    total_time = timer() - overall_start
    print(
        f'\nBest epoch: {best_epoch}, loss: {valid_loss_min:.2f}, accuracy: {100 * valid_acc:.2f}%'
    )
    print(
        f'total training time(s){total_time:.2f}. Time per epoch{total_time / (epoch):.2f} seconds per epoch.'
    )
    # Format history
    history = pd.DataFrame(
        history,
        columns=['train_loss', 'valid_loss', 'train_acc', 'valid_acc'])
    return model, history

train_on_gpu = True
model, history = train(
    model,
    criterion,
    optimizer,
    dataloaders['train'],
    dataloaders['val'],
    save_file_name=save_file_name,
    max_epochs_stop=5,
    n_epochs=40)

# plt.figure(figsize=(16, 12))
fig, ax1 = plt.subplots()
color = 'tab:red'
for c in ['train_loss', 'valid_loss']:
    ax1.plot(
        history[c], label=c)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Average Cross Entropy')
ax1.tick_params(axis='y', labelcolor=color)


ax2 = ax1.twinx()
color = 'tab:blue'
# for c in ['train_acc', 'valid_acc']:
ax2.plot(history['train_acc'], label='train_acc', color='r')
ax2.plot(history['valid_acc'], label='valid_acc', color='g')
ax2.set_ylabel('Average Accuracy')
ax2.tick_params(axis='y', labelcolor=color)

ax1.legend(loc = 10)
ax2.legend(loc = 7)
# plt.xlabel('Epoch')
# plt.ylabel('Average Cross Entropy')
plt.title('Training and Validation Losses & Accuracy')
